In [4]:
import os
import pickle

import dgl

import evaluation
import layers
import numpy as np
import sampler as sampler_module
import torch
import torch.nn as nn
import torchtext
import tqdm
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from model import *
import json

from evaluation import *

import pandas as pd

In [5]:
#Load dataset
dataset_path = "./data/processed_pinsage_Amazon_electronics.pkl"
with open(dataset_path, "rb") as f:
	dataset = pickle.load(f)

In [6]:
user_df = pd.DataFrame(dataset['user-list'])

In [7]:
dataset['full-graph']

Graph(num_nodes={'product': 101128, 'user': 974207},
      num_edges={('product', 'reviewed-by', 'user'): 1744048, ('user', 'reviewed', 'product'): 1744048},
      metagraph=[('product', 'user', 'reviewed-by'), ('user', 'product', 'reviewed')])

In [8]:
# Load config
config_path = "./config/pinsage-params.json"
with open(config_path, "rb") as f:
    model_config = json.load(f)

In [9]:
checkpoint = torch.load("saved_models/price_niche_rating_store/pinsage_AmazonElectronics_500_price_niche_rating_store.pth")

In [10]:
checkpoint['loss']

0.01346542608882828

In [11]:
checkpoint['hit_rate']

0.00396630284939443

In [12]:
h_items = checkpoint['item_embeddings']

In [13]:
g = dataset["train-graph"]
val_matrix = dataset["val-matrix"].tocsr()
test_matrix = dataset["test-matrix"].tocsr()
item_texts = dataset["item-texts"]
user_ntype = dataset["user-type"]
item_ntype = dataset["item-type"]
user_to_item_etype = dataset["user-to-item-type"]
timestamp = dataset["timestamp-edge-column"]

In [14]:
product_list = dataset['product-list']

In [15]:
data = {
    'asin_list': product_list,
    'embeddings': h_items
}

In [16]:
with open('./saved_models/price_niche_ratings/embeddings.pkl', 'wb') as f:
	pickle.dump(data, f)

In [17]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss', 'hit_rate', 'item_embeddings', 'k', 'batch_size'])

In [18]:
def knn_search(query_embedding, node_embeddings, k=10):
    # Compute pairwise Euclidean distances
    distances = torch.cdist(query_embedding.unsqueeze(0), node_embeddings)

    # Sort the distances and get the indices of the K nearest neighbors
    _, indices = torch.topk(distances, k+1, largest=False)

    return indices.squeeze()

In [19]:
recs = knn_search(h_items[23812], h_items)
recs_list = []
idx = 0
for i in list(recs):
	recs_list.append((idx, int(i), 'https://www.amazon.com/dp/' + dataset['product-list'][i]))
	idx += 1
recs_list

[(0, 23812, 'https://www.amazon.com/dp/B094NX9F6C'),
 (1, 65223, 'https://www.amazon.com/dp/B07ZCC8TZJ'),
 (2, 52700, 'https://www.amazon.com/dp/B009KMYJZQ'),
 (3, 64404, 'https://www.amazon.com/dp/B00UWB9XTQ'),
 (4, 42045, 'https://www.amazon.com/dp/B0825M82LQ'),
 (5, 44850, 'https://www.amazon.com/dp/B09PY7KTVG'),
 (6, 1394, 'https://www.amazon.com/dp/B09L4SDN3Q'),
 (7, 4124, 'https://www.amazon.com/dp/B00N8OH1TI'),
 (8, 89942, 'https://www.amazon.com/dp/B0827WZNC9'),
 (9, 34026, 'https://www.amazon.com/dp/B07JN911ST'),
 (10, 17609, 'https://www.amazon.com/dp/B09ZGXQ3DM')]